In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('float_format', '{:f}'.format)

In [4]:
tf.__version__

'1.13.1'

In [2]:
!wget --header="Host: uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/74.0.3729.169 Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en,bg;q=0.9,es;q=0.8,de;q=0.7,fr;q=0.6,it;q=0.5,ja;q=0.4,pt;q=0.3,ru;q=0.2,sr;q=0.1,vi;q=0.1,el;q=0.1" --header="Referer: https://recsys.trivago.cloud/" "https://uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com/zip_by_token_key?dl=1&key=AilA9SlbHXj16VWMNJQyoa9w-L0TnjlKhCmezD8W46gwz0pBT2R7q6kP4Mja95PKD7aT2NdCO3B0Uy6dRK7EPi-sIJYYKZjP1jkRnlIrgyBwLdXJhdhrx3qc8QYBGIuDnnkFa6watmY4GEpnzh7eXBOdILeGVNdYo3mp9HODIjF1bHHfzMJIVJjX1h7ksBmyay8xyx5lE8utbI4NbgkR_M7tvJU8OPvOjxh3k7rmFJaIKcg3RnN5YpHwiKnKYvNzPAf1mglE14GHhsdCxwCrHkgCLZMJvhMnhvCyHjKVM7NW2onjo09ENsHttSoMy1-PC0U" -O "trivagoRecSysChallengeData2019_v2.zip" -c

--2019-06-10 22:50:21--  https://uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com/zip_by_token_key?dl=1&key=AilA9SlbHXj16VWMNJQyoa9w-L0TnjlKhCmezD8W46gwz0pBT2R7q6kP4Mja95PKD7aT2NdCO3B0Uy6dRK7EPi-sIJYYKZjP1jkRnlIrgyBwLdXJhdhrx3qc8QYBGIuDnnkFa6watmY4GEpnzh7eXBOdILeGVNdYo3mp9HODIjF1bHHfzMJIVJjX1h7ksBmyay8xyx5lE8utbI4NbgkR_M7tvJU8OPvOjxh3k7rmFJaIKcg3RnN5YpHwiKnKYvNzPAf1mglE14GHhsdCxwCrHkgCLZMJvhMnhvCyHjKVM7NW2onjo09ENsHttSoMy1-PC0U
Resolving uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com (uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com)... 162.125.69.6, 2620:100:6025:6::a27d:4506
Connecting to uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com (uc31d1308a77caa89e0bfea50ff4.dl.dropboxusercontent.com)|162.125.69.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘trivagoRecSysChallengeData2019_v2.zip’

trivagoRecSysChalle     [              <=>   ] 465,41M  2,29MB/s    in 7m 5s   

2019-06-10 22:57:29 

In [4]:
!unzip trivago* 

Archive:  trivagoRecSysChallengeData2019_v2.zip
mapname:  conversion of  failed
  inflating: train.csv               
  inflating: test.csv                
  inflating: item_metadata.csv       
  inflating: submission_popular.csv  


In [9]:
%%time
df_train = pd.read_csv('train.csv')
df_meta = pd.read_csv('item_metadata.csv')
df_test = pd.read_csv('test.csv')

CPU times: user 38 s, sys: 2.36 s, total: 40.3 s
Wall time: 40.3 s


In [213]:
df_meta.sample(3)

,item_id,properties
634495,48594,Childcare|Radio|4 Star|From 4 Stars|Sitting Ar...
637208,7013756,Luxury Hotel|Hotel
392689,5556348,Restaurant|Free WiFi (Combined)|Free WiFi (Roo...


In [214]:
df_train.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


### Reference valued

In [72]:
sfp = df_train[df_train['action_type'] == 'search for poi']['reference'].unique()
len(sfp), sfp[:3]

(13352, array(['Newtown', 'Surry Hills', 'Covent Garden'], dtype=object))

In [73]:
sfd = df_train[df_train['action_type'] == 'search for destination']['reference'].unique()
len(sfd), sfd[:3]

(22844, array(['London, United Kingdom', 'Bakersfield, USA',
        'Panama City, Panama'], dtype=object))

In [76]:
fs = df_train[df_train['action_type'] == 'filter selection']['reference'].unique()
len(fs), fs[:3]

(201, array(['Very Good Rating', 'Hostal (ES)', 'Hotel'], dtype=object))

In [100]:
coso = df_train[df_train['action_type'] == 'change of sort order']['reference'].unique()
len(coso), coso[:3]

(8, array(['interaction sort button', 'price and recommended', 'price only'],
       dtype=object))

### Meta valued

In [196]:
def get_meta_values(action_type):
    refs = df_train[df_train['action_type'] == action_type]['reference'].unique()
    ids = {int(i) for i in refs if i.isdigit()}
    props = df_meta[df_meta['item_id'].isin(ids)]['properties'].apply(lambda x: x.split('|'))
    return {i for items in props for i in items}

In [198]:
sfi = get_meta_values('search for item')
len(sfi), list(sfi)[:3]

(157, ['Radio', 'Convention Hotel', 'Playground'])

In [199]:
iid = get_meta_values('interaction item deals')
len(iid), list(iid)[:3]

(157, ['Radio', 'Convention Hotel', 'Playground'])

In [200]:
iii = get_meta_values('interaction item image')
len(iii), list(iii)[:3]

(157, ['Radio', 'Convention Hotel', 'Playground'])

In [202]:
iiinf = get_meta_values('interaction item info')
len(iiinf), list(iiinf)[:3]

(157, ['Radio', 'Convention Hotel', 'Playground'])

In [205]:
iir = get_meta_values('interaction item rating')
len(iir), list(iir)[:3]

(157, ['Radio', 'Convention Hotel', 'Playground'])

### Clickout item

In [212]:
df_train[df_train['action_type'] == 'clickout item'].head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...


---

In [221]:
df_train.groupby('session_id')['step'].count().describe()

count   910683.000000
mean        17.495651
std         48.181687
min          1.000000
25%          2.000000
50%          4.000000
75%         13.000000
max       3522.000000
Name: step, dtype: float64

In [245]:
def extract_seqs(group):
    group['session_id']
    group['action_type']

In [227]:
groups = df_train.groupby('session_id')

In [232]:
df = df_train.sample(frac=0.01)

In [233]:
len(df)

159330

In [241]:
groups = df.groupby('session_id')

In [250]:
%%time
train = pd.DataFrame({
    'action_type_seq': groups.apply(lambda x: x['action_type'].tolist()),
    'reference_seq': groups.apply(lambda x: x['reference'].tolist()),
}).reset_index()

CPU times: user 8.68 s, sys: 3.24 ms, total: 8.69 s
Wall time: 8.69 s


In [254]:
train[:10]

,session_id,action_type_seq,reference_seq
0,00000510f1adc,[clickout item],[7281198]
1,000066611146f,[clickout item],[5479306]
2,00019f78d2c36,[interaction item image],[4755476]
3,0001e610ce498,"[interaction item image, interaction item image]","[1257216, 488901]"
4,0001fe0007954,[change of sort order],[price and recommended]
5,00030305c6aa7,[filter selection],[Hostal (ES)]
6,00033ccbbcdd4,[interaction item image],[3854580]
7,00040aa1d0a4d,[interaction item image],[104874]
8,0005174c23dcb,[interaction item info],[1134551]
9,000543ecdcf0f,"[interaction item image, interaction item imag...","[3758872, 3758872, 124561, 3758872]"


In [263]:
impr = groups.apply(lambda x: x[~x['impressions'].isna()]['impressions'].tolist())

In [265]:
impr

session_id
00000510f1adc    [2661832|9222426|7051844|4079190|5752778|46839...
000066611146f    [5479306|97126|2195126|4163966|8415348|7829686...
00019f78d2c36                                                   []
0001e610ce498                                                   []
0001fe0007954                                                   []
00030305c6aa7                                                   []
00033ccbbcdd4                                                   []
00040aa1d0a4d                                                   []
0005174c23dcb                                                   []
000543ecdcf0f                                                   []
00054e0206883                                                   []
0005c6da8ea8a    [1499679|1282296|1923369|5097960|152217|471526...
000667c53544d                                                   []
0006c76c444d6                                                   []
000740962ae8f                                      